# Decision Tree Implementation from Scratch

In [1]:
import pandas as pd
import numpy as np

## Dataset

In [2]:
X = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y = np.array([1,1,0,0,1,0,0,1,1,0])

X_train  = pd.DataFrame(X ,columns=['Brown_Cap','Tapering_Stalk_Shape','Solitary'],index=[0,1,2,3,4,5,6,7,8,9])
y_train  = pd.DataFrame(y ,columns=['Edible'],index=[0,1,2,3,4,5,6,7,8,9])

frames = [X_train , y_train]
dataset = pd.concat(frames,axis=1)
dataset.head(10)

,Brown_Cap,Tapering_Stalk_Shape,Solitary,Edible
0,1,1,1,1
1,1,0,1,1
2,1,0,0,0
3,1,0,0,0
4,1,1,1,1
5,0,1,1,0
6,0,0,0,0
7,1,0,1,1
8,0,1,0,1
9,1,0,0,0


## Model

in a decision tree, we decide if a node will be split or not by looking at the **information gain** that split would give us. 
Where 

$$\text{Information Gain} = H(p_1^\text{node})- \left(w^{\text{left}}H\left(p_1^\text{left}\right) + w^{\text{right}}H\left(p_1^\text{right}\right)\right),$$

and $H$ is the entropy, defined as

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$

weighted entropy, defined as

$$\text{weighted entropy} = \left (w^{\text{left}}H\left(p_1^\text{left}\right) + w^{\text{right}}H\left(p_1^\text{right}\right)\right)$$

So, the root node has every animal in our dataset. Remember that $p_1^{node}$ is the proportion of positive class (cats) in the root node.

In [3]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
def entropy(y):
    entropy = 0
    
    if len(y) != 0:
        p = len(y[y==1]) / len(y)       
        if p == 0 or p == 1:  entropy =  0
        else: entropy = -p * np.log2(p) - ((1-p) * np.log2(1-p))
            
    return entropy

In [5]:
def split_indices(X, node_indices,feature):
    left_indices = [] 
    right_indices = []
    
    for i in node_indices:
        if X[i][feature] == 1: left_indices.append(i)
        else: right_indices.append(i)
    
    return left_indices, right_indices

In [6]:
def information_gain(X,y,node_indices, feature):
    left_indices, right_indices = split_indices(X, node_indices, feature)
    
    y_node = y[node_indices]
    y_left = y[left_indices]
    y_right = y[right_indices]
    
    node_entropy = entropy(y_node)
    left_entropy = entropy(y_left)
    right_entropy = entropy(y_right)
    
    w_left = len(y_left) / len(y_node)
    w_right = len(y_right) / len(y_node)
    
    weighted_entropy = w_left * left_entropy + w_right * right_entropy
    information_gain = node_entropy - weighted_entropy
    
    return information_gain

In [7]:
def get_best_split(X,y,node_indices):
    num_features = X.shape[1]
    best_feature = -1
    max_info_gain = 0
    
    for feature in range(num_features):
        gain = information_gain(X, y,node_indices, feature)
        
        if gain > max_info_gain:  
            max_info_gain = gain
            best_feature = feature 

    return best_feature

In [8]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    # stopping criteria
    if current_depth == max_depth:
        print( branch_name ," leaf node with indices " , node_indices)
        return
    
    best_feature = get_best_split(X, y, node_indices) 
    print("Depth %d, %s: Split on feature: %d" % (current_depth, branch_name, best_feature))
    
    left_indices, right_indices = split_indices(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [9]:
build_tree_recursive(X, y, root_indices, "Root", 2, 0)

Depth 0, Root: Split on feature: 2
Depth 1, Left: Split on feature: 0
Left  leaf node with indices  [0, 1, 4, 7]
Right  leaf node with indices  [5]
Depth 1, Right: Split on feature: 1
Left  leaf node with indices  [8]
Right  leaf node with indices  [2, 3, 6, 9]


In [10]:
def predict(x):      
    if x[2] == 1:
        if x[0] == 1: return 1
        elif x[0] == 0: return 0
        
    elif x[2] == 0:
        if x[1] == 1: return 1
        elif x[1] == 0: return 0
        
print(predict([1,1,1]))
print(predict([1,0,1]))
print(predict([1,0,0]))
print(predict([0,1,1]))
print(predict([1,0,0]))

1
1
0
0
0


# Using scikit learn

In [11]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred_clf = clf.score(X_train , y_train)
print(y_pred_clf)

1.0
